In [ ]:
!pip install pandas
!pip install numpy==1.23

In [ ]:
!pip install tensorflow==2.12
!pip install pyarabic

In [ ]:
!pip install transformers

In [ ]:
!pip install ipywidgets
!pip install datasets
!pip install transformers[torch]
!pip install nvidia-ml-py3

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)

2023-07-27 08:34:23.581089: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 08:34:23.687709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 08:34:24.407514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from tqdm import tqdm
from transformers import BertTokenizerFast, AutoTokenizer
from datasets import load_dataset
import glob

folder_path = 'text/'
files_list = glob.glob(folder_path + "/*")
dataset = load_dataset('text', data_files=files_list)

Found cached dataset text (/home/ffq/.cache/huggingface/datasets/text/default-daa5a8d92e7d97fb/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 127836649
})

In [8]:
from tqdm import tqdm
from transformers import BertTokenizerFast, AutoTokenizer
from datasets import load_dataset
import glob

#loading bert tokenizer to work as a base for the new tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

folder_path = 'text/'
files_list = glob.glob(folder_path + "/*")
dataset = load_dataset('text', data_files=files_list)

def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(dataset['train']), batch_size)):
        yield dataset['train'][i: i +batch_size]['text']
bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), 
                                                   vocab_size=50000
                                                   #, special_tokens=['[CLS]', '[PAD]','[SEP]','[UNK]','[MASK]']
                                                  )
bert_tokenizer.save_pretrained('bert_bbpe/')

Found cached dataset text (/home/ffq/.cache/huggingface/datasets/text/default-daa5a8d92e7d97fb/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████| 12784/12784 [06:02<00:00, 35.25it/s]


('bert_bbpe/tokenizer_config.json',
 'bert_bbpe/special_tokens_map.json',
 'bert_bbpe/vocab.json',
 'bert_bbpe/merges.txt',
 'bert_bbpe/added_tokens.json',
 'bert_bbpe/tokenizer.json')

In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 127836649
    })
})

In [1]:
# cell-3
#tokenizing the whole text 

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from datasets import load_dataset
import glob
import tokenizers
from transformers import Trainer, TrainingArguments, LineByLineTextDataset, BertModel
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('faisalq/bert-base-arapoembert')
tokenizer = AutoTokenizer.from_pretrained('bert_bbpe/')
max_seq_length = 128

folder_path = 'text/'
files_list = glob.glob(folder_path + "/*")
dataset = load_dataset('text', data_files=files_list)

# use to combine files: cat ./* > merged_file

def encode_with_truncation(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_seq_length, return_special_tokens_mask=True)


dataset = dataset["train"].map(encode_with_truncation, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# dataset = LineByLineTextDataset(tokenizer = tokenizer, file_path = 'text/text4.txt', 
#                                block_size = max_seq_length)
display(len(dataset))

2023-08-22 17:33:41.779230: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-22 17:33:41.802559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 17:33:42.326262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Found cached dataset text (/home/ffq/.cache/huggingface/datasets/text/default-daa5a8d92e7d97fb/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/127836649 [00:00<?, ? examples/s]

127836649

In [2]:
dataset.features

{'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [2]:
dataset = dataset.remove_columns(["text"])

In [3]:
dataset.save_to_disk("tokenized_dataset_bbpe/dataset")

Saving the dataset (0/200 shards):   0%|          | 0/127836649 [00:00<?, ? examples/s]

In [1]:
#start here

from datasets import load_from_disk

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from datasets import load_dataset
import glob
import tokenizers
from transformers import Trainer, TrainingArguments, LineByLineTextDataset, BertModel
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling
from transformers import AutoTokenizer

dataset = load_from_disk("tokenized_dataset_bbpe/dataset")

2023-08-26 11:21:35.970763: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-26 11:21:36.306548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 11:21:36.881774: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# cell-4
# model config
max_seq_length = 128
tokenizer = AutoTokenizer.from_pretrained('bert_bbpe/')
config = BertConfig( vocab_size = 50000, 
                    hidden_size = 768, 
                    num_hidden_layers = 12,
                    num_attention_heads = 12,
                    max_position_embeddings = max_seq_length)

model = BertForMaskedLM(config)
display(model.num_parameters())

124197968

In [3]:
# cell-5
#pretraining the model

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True,
                                               mlm_probability=0.15)
epochs = 2
save_steps = 10000 #save checkpoint every 10000 steps
batch_size = 128

training_args = TrainingArguments(
    output_dir = 'bert_bbpe/',
    overwrite_output_dir=True,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    save_steps = save_steps,
    save_total_limit = 5, #only save the last 5 checkpoints
    fp16=True,
    # tf32 = True,
    learning_rate = 5e-5,  # 5e-5 is the default
    logging_steps = 5_000,
    # gradient_accumulation_steps=4,
    # gradient_checkpointing=True
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset=dataset

)

from pynvml import *


trainer.train(resume_from_checkpoint=True)
# trainer.train()
trainer.save_model('bert_bbpe/')

/home/ffq/miniconda3/envs/p1/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1205000,2.642800
1210000,2.642900
1215000,2.635000
1220000,2.637200
1225000,2.636600
1230000,2.632700
1235000,2.632200
1240000,2.630300
1245000,2.630200
1250000,2.628500
